 # Titanic: Machine Learning from Disaster
 
 https://www.kaggle.com/c/titanic

## Load required packages

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, 
    learning_curve, validation_curve, GridSearchCV
)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostClassifier

import xgboost as xgb
from xgboost import XGBClassifier

## Load dataset

In [2]:
dataset = pd.read_csv("train.csv")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Data Dictionary

Variable	Definition	
survival 	Survival 	
pclass 	Ticket class 
sex 	Sex 	
Age 	Age in years 	
sibsp 	# of siblings / spouses aboard the Titanic 	
parch 	# of parents / children aboard the Titanic 	
ticket 	Ticket number 	
fare 	Passenger fare 	
cabin 	Cabin number 	
embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton

## Data Preprocessing

In [3]:
y = dataset["Survived"]
X = pd.DataFrame(dataset[dataset.columns.difference(["PassengerId", "Cabin", "Ticket"])])
X.head()

,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
0,22.0,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0
1,38.0,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1
2,26.0,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1
3,35.0,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1
4,35.0,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0


### Feature engineering

#### Age

In [4]:
X["Age.agr"] = X["Age"].map(lambda x: round(x,0) // 10)

In [5]:
X.groupby(by=["Age.agr", "Sex"]).agg({"Survived": ["mean", "count"]})

Survived      
                    mean count
Age.agr Sex                   
0.0     female  0.633333    30
        male    0.593750    32
1.0     female  0.755556    45
        male    0.122807    57
2.0     female  0.722222    72
        male    0.168919   148
3.0     female  0.833333    60
        male    0.214953   107
4.0     female  0.687500    32
        male    0.210526    57
5.0     female  0.888889    18
        male    0.133333    30
6.0     female  1.000000     4
        male    0.133333    15
7.0     male    0.000000     6
8.0     male    1.000000     1

In [6]:
def age_mapper(x):
    if pd.isnull(x):
        return "unknown"
    if x < 10:
        return "child"
    return "adult"

X["Age.cut"] = X["Age"].map(age_mapper)
X.groupby(by=["Age.cut", "Sex"]).agg({"Survived": ["mean", "count"]})

Survived      
                    mean count
Age.cut Sex                   
adult   female  0.770563   231
        male    0.175772   421
child   female  0.633333    30
        male    0.593750    32
unknown female  0.679245    53
        male    0.129032   124

#### Name

In [7]:
X["Title"] = X["Name"].map(lambda x: x.split(",")[-1].strip().split(" ")[0])
X.groupby(by="Title").agg({"Survived": ["mean", "size"]})

Survived     
               mean size
Title                   
Capt.      0.000000    1
Col.       0.500000    2
Don.       0.000000    1
Dr.        0.428571    7
Jonkheer.  0.000000    1
Lady.      1.000000    1
Major.     0.500000    2
Master.    0.575000   40
Miss.      0.697802  182
Mlle.      1.000000    2
Mme.       1.000000    1
Mr.        0.156673  517
Mrs.       0.792000  125
Ms.        1.000000    1
Rev.       0.000000    6
Sir.       1.000000    1
the        1.000000    1

In [8]:
def title_mapper(title):
    if title in ("Miss.", "Mille."):
        return "Miss"
    if title in ("Mr.",):
        return "Mr."
    if title in ("Mrs."):
        return "Mrs."
    return "Other"

X["Title"] = X["Title"].map(title_mapper)
X.groupby(by="Title").agg({"Survived": ["mean", "count"]})

Survived      
           mean count
Title                
Miss   0.697802   182
Mr.    0.156673   517
Mrs.   0.792000   125
Other  0.522388    67

#### Family Size

In [9]:
X["FamilySize"] = X["SibSp"] + X["Parch"] + 1
X.groupby(by="FamilySize").agg({"Survived": ["mean", "count"]})

Survived      
                mean count
FamilySize                
1           0.303538   537
2           0.552795   161
3           0.578431   102
4           0.724138    29
5           0.200000    15
6           0.136364    22
7           0.333333    12
8           0.000000     6
11          0.000000     7

In [10]:
def familysize_mapper(x):
    if x == 1:
        return "singleton"
    if x < 5:
        return "small"
    return "large"

X["FamilySize"] = X["FamilySize"].map(familysize_mapper)
X.groupby(by="FamilySize").agg({"Survived": ["mean", "count"]})

Survived      
                mean count
FamilySize                
large       0.161290    62
singleton   0.303538   537
small       0.578767   292

### Imput missing values

In [11]:
X.isna().sum()

Age           177
Embarked        2
Fare            0
Name            0
Parch           0
Pclass          0
Sex             0
SibSp           0
Survived        0
Age.agr       177
Age.cut         0
Title           0
FamilySize      0
dtype: int64

In [12]:
X[["Embarked"]] = X[["Embarked"]].fillna(value=X["Embarked"].value_counts().index[0])

age_imputer = Imputer(missing_values="NaN", strategy="mean", axis=0).fit(X[["Age"]])
X[["Age"]] = age_imputer.transform(X[["Age"]])

### Handling categorical data

In [13]:
X = pd.get_dummies(X, columns=["Pclass", "Embarked", "Sex", "Age.cut", "FamilySize", "Title"], drop_first=True)
X = X.drop(["Age.agr", "Name", "Survived"], axis=1)
X.head()

,Age,Fare,Parch,SibSp,Pclass_2,Pclass_3,Embarked_Q,Embarked_S,Sex_male,Age.cut_child,Age.cut_unknown,FamilySize_singleton,FamilySize_small,Title_Mr.,Title_Mrs.,Title_Other
0,22.0,7.2500,0,1,0,1,0,1,1,0,0,0,1,1,0,0
1,38.0,71.2833,0,1,0,0,0,0,0,0,0,0,1,0,1,0
2,26.0,7.9250,0,0,0,1,0,1,0,0,0,1,0,0,0,0
3,35.0,53.1000,0,1,0,0,0,1,0,0,0,0,1,0,1,0
4,35.0,8.0500,0,0,0,1,0,1,1,0,0,1,0,1,0,0


### Create training and test sets

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [15]:
print("Train split: ", np.bincount(y_train)/y_train.shape[0])
print("Test split: ", np.bincount(y_test)/y_test.shape[0])

Train split:  [0.61637239 0.38362761]
Test split:  [0.61567164 0.38432836]


### Standardize features

In [16]:
class SubsetStandardScaler(StandardScaler):
    
    def __init__(self, columns=None, **kwargs):
        super().__init__(**kwargs)
        self.columns = columns
        
    def fit(self, X, y=None):
        if self.columns is None:
            return super().fit(X, y)
        return super().fit(X[self.columns], y)
        
    def transform(self, X, y="deprecated", copy=None):
        if self.columns is None:
            return super().transform(X, y, copy)
        X_std = X.copy()
        X_subset_std = super().transform(X_std[self.columns], y, copy)
        for i in range(X_subset_std.shape[1]):
            X_std[[self.columns[i]]] = X_subset_std[:,i]
        return X_std.values

## Algorithm Selection

Using nested cross-validation select the best algorithm among following:
- logistic regression
- svc
- decision tree classifier
- random forests
- k-nearest neighbors

### Logistic Regression

In [186]:
pipe_lr = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    LogisticRegression()
)

gs = GridSearchCV(
    estimator=pipe_lr,
    param_grid = [ { "logisticregression__C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]} ],
    scoring="accuracy", cv=2
)

scores = cross_val_score(gs, X_train, y_train, scoring="accuracy", cv=5)

print("CV accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

CV accuracy: 0.811 +/- 0.018


### SVC

In [187]:
pipe_svc = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    SVC(random_state=1)
)

param_range = [ 10**i for i in range(-2, 2,) ]
param_grid = [
    {"svc__C": param_range, "svc__kernel": ["linear"]},
    {"svc__C": param_range, "svc__gamma": param_range, "svc__kernel": ["rbf"]}
]

gs = GridSearchCV(
    estimator=pipe_svc, param_grid=param_grid, scoring="accuracy", 
    cv=2, n_jobs=-1
)
scores = cross_val_score(gs, X_train, y_train, scoring="accuracy", cv=5)

print("CV accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

CV accuracy: 0.822 +/- 0.009


### Decision Tree Classifier

In [188]:
dtc_pipe = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    DecisionTreeClassifier(random_state=1)
)

gs = GridSearchCV(
    estimator=dtc_pipe,
    param_grid=[{ "decisiontreeclassifier__max_depth": [1, 2, 3, 4, 5, 6, 7, None]}],
    scoring="accuracy", cv=2, n_jobs=-1
)
scores = cross_val_score(gs, X_train, y_train, scoring="accuracy", cv=5)

print("CV accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

CV accuracy: 0.790 +/- 0.020


### Random Forest

In [189]:
rf_pipe = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    RandomForestClassifier(random_state=1)
)

gs = GridSearchCV(
    estimator=rf_pipe,
    param_grid=[{
        "randomforestclassifier__max_depth": [1, 2, 3, None],
        "randomforestclassifier__n_estimators": [10, 50, 100, 150, 200, 250]
    }],
    cv=2, n_jobs=-1, scoring="accuracy"
)
scores = cross_val_score(gs, X_train, y_train, scoring="accuracy", cv=5)

print("CV accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

CV accuracy: 0.806 +/- 0.012


#### Features importance

In [197]:
forest = RandomForestClassifier(n_estimators=500, random_state=1).fit(X_train, y_train)

In [207]:
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

feat_labels = X_train.columns

for i in range(X_train.shape[1]):
    print(feat_labels[indices[i]], importances[indices[i]])

Fare 0.25243828612305524
Age 0.1973139857118892
Title_Mr. 0.1394167310244178
Sex_male 0.1317164549785633
Pclass_3 0.061058640450614494
SibSp 0.038327996579937657
Title_Mrs. 0.028176646443386036
Parch 0.025181795635316067
FamilySize_small 0.023169679961288468
Embarked_S 0.022625594288312376
Pclass_2 0.01846732778161857
Age.cut_unknown 0.016485933869524787
Title_Other 0.012303052606015373
FamilySize_singleton 0.012025843649076223
Age.cut_child 0.011694772660538772
Embarked_Q 0.00959725823644563


### K-nearest neighbors

In [190]:
knn_pipe = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    KNeighborsClassifier(p=2, metric="minkowski")
)

gs = GridSearchCV(
    estimator=knn_pipe,
    param_grid=[{
        "kneighborsclassifier__n_neighbors": [2, 5, 10, 15, 20, 25]
    }],
    scoring="accuracy", cv=2, n_jobs=-1
)
scores = cross_val_score(gs, X_train, y_train, scoring="accuracy", cv=5)

print("CV accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

CV accuracy: 0.782 +/- 0.015


SVC with accuracy of 0.825 is the best algorithm.

## SVC - Model Selection

In [191]:
pipe_svc = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    SVC(random_state=1)
)

param_range = [ 10**i for i in range(-2, 2,) ]
param_grid = [
    {"svc__C": param_range, "svc__kernel": ["linear"]},
    {"svc__C": param_range, "svc__gamma": param_range, "svc__kernel": ["rbf"]}
]

gs = GridSearchCV(
    estimator=pipe_svc, param_grid=param_grid, scoring="accuracy", 
    cv=2, n_jobs=-1
).fit(X_train, y_train)

In [192]:
gs.best_params_

{'svc__C': 10, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

In [193]:
pipe_svc = make_pipeline(
    SubsetStandardScaler(columns=["Age", "Fare"]),
    SVC(kernel="rbf", gamma=0.01, C=1)
).fit(X_train, y_train)

In [194]:
print("SVC accuracy on test set: %.3f" % pipe_svc.score(X_test, y_test))

SVC accuracy on test set: 0.821


## Ensemble Methods

### AdaBoost

In [35]:
stump = DecisionTreeClassifier(criterion="entropy", random_state=1, max_depth=1)

ada = AdaBoostClassifier(base_estimator=stump, n_estimators=1000, learning_rate=1, random_state=1)

In [48]:
ada_params = [{ 
    "n_estimators": [100 + i*100 for i in range(10) ],
    "learning_rate": [ 0.1 + i*0.1 for i in range(10) ] 
}]

ada_gs = GridSearchCV(
    estimator=ada, param_grid=ada_params, 
    scoring="accuracy", cv=2, n_jobs=-1
).fit(X_train, y_train)

In [53]:
print(ada_gs.best_params_)
ada = ada_gs.best_estimator_

{'learning_rate': 0.1, 'n_estimators': 100}


In [55]:
ada.score(X_test, y_test)

0.8134328358208955

### XGBoost - Extreme Gradient Boosting

In [27]:
X_train.columns

Index(['Age', 'Fare', 'Parch', 'SibSp', 'Pclass_2', 'Pclass_3', 'Embarked_Q',
       'Embarked_S', 'Sex_male', 'Age.cut_child', 'Age.cut_unknown',
       'FamilySize_singleton', 'FamilySize_small', 'Title_Mr.', 'Title_Mrs.',
       'Title_Other'],
      dtype='object')

In [48]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [80]:
xgb_params = {
    'max_depth': 2, 'eta': 0.01, 'silent': 1, 'objective': 'binary:logistic',
    "nthread": 4, "eval_metric": "auc", "n_estimators": 10000
}

bst = xgb.train(xgb_params, dtrain, 10)

In [81]:
y_train_pred = bst.predict(dtrain) > 0.5
y_test_pred = bst.predict(dtest) > 0.5

In [82]:
print("XGBOost accuracy on test set: %.3f" % accuracy_score(y_test, y_test_pred))

XGBOost accuracy on test set: 0.776
